# MTG Format Price Standard

Magic: the Gathering has a variety of platforms where a player can choose to acquire cards.  The following data is sourced from MTGGoldfish.com, where they perform analyses of format metagames, and Scryfall.com, which hosts an exhaustive database of all MTG cards.  The goal of this project was to determine what popular Standard Decks cost on each platform so that a prospective player of the format can understand at a glance where they might want to invest their money.  There are other considerations when considering a platform to play on or deck to play, but an at-a-glance analysis of prices by platform is valuable.  Furthermore, many deckbuilding sites have information about paper and MTGO prices, but the idea of calculating a price for a deck in USD for MTG Arena is novel, and I developed a method for calculating this based on the store prices and information available about the in-game economy.

In [7]:
import pandas as pd
import json
import requests

The following cell gets a list of all oracle cards from scryfall, normalizes the dataset, puts away columns that we don't need for this analysis, renames the other columns, and then keeps only standard legal cards.  The final result is a nice dataframe of all the cards legal in standard alongside their rarity and prices, ready to be used as reference for deck price analysis.

In [8]:
bulk = requests.get('https://api.scryfall.com/bulk-data') #I learned that the uri for bulk data changes daily, so requesting the bulk data for the day then getting the uri for updated cards is needed (I had outdated data)
bulk_df = pd.json_normalize(bulk.json(),'data')
oracle_uri = bulk_df['download_uri'].loc[bulk_df.query('type == "oracle_cards"').index[0]]

response = requests.get(oracle_uri)
raw_oracle_cards = pd.json_normalize(response.json())

oracle_cards = raw_oracle_cards[[#'object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id',
       #'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 
       'name', 
       #'lang',
       #'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image',
       #'image_status', 'mana_cost', 'cmc', 'type_line', 'oracle_text',
       #'colors', 'color_identity', 'keywords', 'games', 'reserved',
       #'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint',
       #'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri',
       #'set_search_uri', 'scryfall_set_uri', 'rulings_uri',
       #'prints_search_uri', 'collector_number', 'digital', 
       'rarity',
       #'flavor_text', 'card_back_id', 'artist', 'artist_ids',
       #'illustration_id', 'border_color', 'frame', 'full_art', 'textless',
       #'booster', 'story_spotlight', 'edhrec_rank', 'image_uris.small',
       #'image_uris.normal', 'image_uris.large', 'image_uris.png',
       #'image_uris.art_crop', 'image_uris.border_crop',
       'legalities.standard',
       #'legalities.future', 'legalities.historic',
       #'legalities.gladiator', 'legalities.pioneer',
       #'legalities.explorer', 'legalities.modern', 'legalities.legacy',
       #'legalities.pauper', 'legalities.vintage', 'legalities.penny',
       #'legalities.commander', 'legalities.brawl',
       #'legalities.historicbrawl', 'legalities.alchemy',
       #'legalities.paupercommander', 'legalities.duel',
       #'legalities.oldschool', 'legalities.premodern', 
       'prices.usd',
       #'prices.usd_foil', 'prices.usd_etched', 'prices.eur',
       #'prices.eur_foil', 
       'prices.tix', 
       #'related_uris.gatherer',
       #'related_uris.tcgplayer_infinite_articles',
       #'related_uris.tcgplayer_infinite_decks', 'related_uris.edhrec',
       #'security_stamp', 'preview.source', 'preview.source_uri',
       #'preview.previewed_at', 'power', 'toughness', 'penny_rank',
       #'arena_id', 'watermark', 'produced_mana', 'all_parts',
       'card_faces', 
       #'frame_effects', 'tcgplayer_etched_id',
       #'promo_types', 'loyalty', 'life_modifier', 'hand_modifier',
       #'attraction_lights', 'color_indicator', 'content_warning'
       ]]
oracle_cards = oracle_cards.rename(columns = {
    'name':'Name',
    'rarity':'Rarity',
    'legalities.standard':'Standard_Legal',
    'prices.usd':'Price_USD',
    'prices.tix':'Price_Tix'
})

standard_raw = oracle_cards.query('Standard_Legal == "legal"').sort_values(by =['Name']).reset_index(drop = True)
standard_raw.head()

,Name,Rarity,Standard_Legal,Price_USD,Price_Tix,card_faces
0,A Little Chat,uncommon,legal,0.04,0.01,NaN
1,Abandon the Post,common,legal,0.01,0.03,NaN
2,Abrade,common,legal,0.16,0.03,NaN
3,Academy Loremaster,rare,legal,0.15,0.01,NaN
4,Academy Wall,common,legal,0.05,0.01,NaN


MDFCs, or dual-sided cards, are referenced only by their front-facing name on MTGGoldfish but have both sides included in their main name column from Scryfall.  Scryfall has a 'card_faces' column that includes data on both sides of the card; here I replace the original name in the dataframe with the front-side name found in the card_faces data, then drop the card_face column as it is no longer needed

In [9]:
card_faces = pd.json_normalize(standard_raw['card_faces'].loc[~standard_raw['card_faces'].isna()])
front_face = pd.json_normalize(card_faces[0])
front_face_index = 0
for card_index in standard_raw.loc[~standard_raw['card_faces'].isna()].index:
    standard_raw["Name"].loc[card_index] = front_face['name'].loc[front_face_index]
    front_face_index += 1
standard = standard_raw.drop(['card_faces'],axis=1)
standard.head()

,Name,Rarity,Standard_Legal,Price_USD,Price_Tix
0,A Little Chat,uncommon,legal,0.04,0.01
1,Abandon the Post,common,legal,0.01,0.03
2,Abrade,common,legal,0.16,0.03
3,Academy Loremaster,rare,legal,0.15,0.01
4,Academy Wall,common,legal,0.05,0.01


Now we set up a dataframe to hold info about our decklists found on mtggoldfish.com

In [10]:
decks = pd.DataFrame(columns=['Deck_Name','Deck_List','Paper_Price','MTGO_Price_Tix','Num_Rare','Num_Mythic','MTGArena_Price'])
standard_deck_names = ['Grixis Midrange','Mono-White Midrange','5 Color Midrange','Mono-Red Aggro',
                       'Esper Midrange','Mono-Blue Tempo','Azorius Soldiers','Selesnya Poison',
                       'Jund Midrange','Kami War','Esper Legends','Mardu Midrange','Rakdos Aggro',
                       'Domain Control','Esper Aggro']
decks['Deck_Name'] = standard_deck_names
decks.head(len(standard_deck_names))

,Deck_Name,Deck_List,Paper_Price,MTGO_Price_Tix,Num_Rare,Num_Mythic,MTGArena_Price
0,Grixis Midrange,NaN,NaN,NaN,NaN,NaN,NaN
1,Mono-White Midrange,NaN,NaN,NaN,NaN,NaN,NaN
2,5 Color Midrange,NaN,NaN,NaN,NaN,NaN,NaN
3,Mono-Red Aggro,NaN,NaN,NaN,NaN,NaN,NaN
4,Esper Midrange,NaN,NaN,NaN,NaN,NaN,NaN
5,Mono-Blue Tempo,NaN,NaN,NaN,NaN,NaN,NaN
6,Azorius Soldiers,NaN,NaN,NaN,NaN,NaN,NaN
7,Selesnya Poison,NaN,NaN,NaN,NaN,NaN,NaN
8,Jund Midrange,NaN,NaN,NaN,NaN,NaN,NaN
9,Kami War,NaN,NaN,NaN,NaN,NaN,NaN


Each entry in the Deck_List column is another dataframe containing the decklist for a given deck.  I think there might be a more efficient way to write this bit but it works quickly and achieves what I need, so it's cool

In [11]:
standard_deck_decklists = []
for deck in standard_deck_names:
    standard_deck_decklists.append(pd.read_csv('Deck Lists/Standard Metagame Decks/'+deck+'.csv'))

decks['Deck_List'] = standard_deck_decklists
decks.head(len(standard_deck_names))


,Deck_Name,Deck_List,Paper_Price,MTGO_Price_Tix,Num_Rare,Num_Mythic,MTGArena_Price
0,Grixis Midrange,Quantity Card_Name 0 ...,NaN,NaN,NaN,NaN,NaN
1,Mono-White Midrange,Quantity Card_Name 0 ...,NaN,NaN,NaN,NaN,NaN
2,5 Color Midrange,Quantity Card_Name 0 ...,NaN,NaN,NaN,NaN,NaN
3,Mono-Red Aggro,Quantity Card_Name 0...,NaN,NaN,NaN,NaN,NaN
4,Esper Midrange,Quantity Card_Name 0 ...,NaN,NaN,NaN,NaN,NaN
5,Mono-Blue Tempo,Quantity Card_Name 0 ...,NaN,NaN,NaN,NaN,NaN
6,Azorius Soldiers,Quantity Card_Name 0 ...,NaN,NaN,NaN,NaN,NaN
7,Selesnya Poison,Quantity Card_Name 0 ...,NaN,NaN,NaN,NaN,NaN
8,Jund Midrange,Quantity Card_Name 0...,NaN,NaN,NaN,NaN,NaN
9,Kami War,Quantity Card_Name 0 ...,NaN,NaN,NaN,NaN,NaN


The following are functions that calculate the price of a deck given the decklist from MTGGoldfish using the USD and Tix prices listed on Scryfall (Scryfall uses TCGPlayer and CardHoarder for pricings respectively)

In [12]:
def paper_price(decklist):
    deck_price = 0
    #with the decklist, we want to find the price from out standard dataframe, then multiply it by the quantity of card, then add that to price
    for card_index in range(len(decklist.index)): #iterate through the cards
        #get the individual card names and quantities
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        price_index = standard.query('Name == "'+card_name+'"')['Price_USD'].index[0]
        card_price = float(standard['Price_USD'].loc[price_index])

        #add the price of the card for each time that the card 
        deck_price += (card_price * card_quantity)
        

    return deck_price

In [13]:
def mtgo_price(decklist):
    deck_price = 0
    #with the decklist, we want to find the price from out standard dataframe, then multiply it by the quantity of card, then add that to price
    for card_index in range(len(decklist.index)): #iterate through the cards
        #get the individual card names and quantities
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        price_index = standard.query('Name == "'+card_name+'"')['Price_USD'].index[0]
        if standard['Price_Tix'].loc[price_index]:
            card_price = float(standard['Price_Tix'].loc[price_index])
        else:
            card_price = 0

        #add the price of the card for each time that the card 
        deck_price += (card_price * card_quantity)
        

    return deck_price

This function sums the rares, very similar to above

In [14]:
def count_rares(decklist):
    #we want to count the number of rares and mythics per deck
    num_rare = 0
    num_mythic = 0
    for card_index in range(len(decklist.index)):
        #get card info from decklist
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        rarity_index = standard.query('Name == "'+card_name+'"')['Rarity'].index[0]
        if standard['Rarity'].loc[rarity_index] == 'rare':
            num_rare += card_quantity
        if standard['Rarity'].loc[rarity_index] == 'mythic':
            num_mythic += card_quantity

    return(num_rare,num_mythic)

Price for MTGA is a little more obfuscated due to the options of the in-game economy.  I try to optimize price based on purchasing options for a given amount of rare and mythic cards. This took a handful of iterations but essentially winds up being always purchasing in a way that provides the cheapest card, and doing so in an order that optimizes each successive purchase.  Though probably non-trivial, I am assuming that a player has sufficient wildcards to cover their common and uncommon needs for a given deck.

In [15]:
#for reference, this was a simple attempt to get a baseline idea to optimize a buying strategy against.  no longer used
def buying_wildcards(r,m):
    price = 0
    rare_wildcard_bundles = 0
    mythic_wildcard_bundles = 0
    packs = 0
    #buy as many wildcard packs as you can
    while r>0 and rare_wildcard_bundles <= 10:
        rare_wildcard_bundles += 1
        price += 10
        r-=4
    while m>0 and mythic_wildcard_bundles <= 10:
        mythic_wildcard_bundles += 1
        price += 20
        m -= 4
    #finish with packs
    while (r>0 or m>0):
        packs += 6
        price += 6
        if packs % 30 == 0:
            m -= 1
        else:
            r -= 1

    return price

In [16]:
def best_strategy (r,m):
    price = 0
    rare_wildcard_bundles = 0
    mythic_wildcard_bundles = 0
    packs = 0

     #optimally buy mythics, bundles are ideal until at least m < 4 
    while m >= 4 and mythic_wildcard_bundles <= 10:
        mythic_wildcard_bundles += 1
        price += 10
        m -= 4

    #buy wildcards until less than 2 or out of buys (if there is one left, it costs $6 to get it with packs. if there are two, those two are basiclly $5 plus you get two extra)
    while r >= 2 and rare_wildcard_bundles <= 10:
        rare_wildcard_bundles += 1
        price += 10
        r -= 4

    #finish rares with packs
    while r>0:
        packs += 6
        price += 6
        if packs % 30 == 0:
            m -= 1
        else:
            r -= 1

    if(r>=0 and m >=0):
        return price

    mythic_bundle_price = 20/m
    mythic_packs_price = (30 - packs) + (30*(m-1))

    if mythic_bundle_price < mythic_packs_price:
        mythic_wildcard_bundles += 1
        price += 10
        m -= 4

    if mythic_packs_price < mythic_bundle_price:
        while m > 0:
            packs += 6
            price += 6
            if packs % 30 == 0:
                m -= 1
            else:
                r -= 1

    return price

    return price


In [17]:
for index in decks.index:
    decks['Paper_Price'].loc[index] = paper_price(decks['Deck_List'].loc[index])
    decks['MTGO_Price_Tix'].loc[index] = mtgo_price(decks['Deck_List'].loc[index])
    decks['Num_Rare'].loc[index],decks['Num_Mythic'].loc[index] = count_rares(decks['Deck_List'].loc[index])
    decks['MTGArena_Price'].loc[index] = best_strategy(decks['Num_Rare'].loc[index],decks['Num_Mythic'].loc[index])
decks.sort_values('MTGArena_Price',ascending=False)

,Deck_Name,Deck_List,Paper_Price,MTGO_Price_Tix,Num_Rare,Num_Mythic,MTGArena_Price
4,Esper Midrange,Quantity Card_Name 0 ...,501.96,435.41,57,8,226
14,Esper Aggro,Quantity Card_Name 0 ...,451.96,364.99,55,9,208
10,Esper Legends,Quantity Card_Name 0 ...,589.07,571.16,52,8,184
6,Azorius Soldiers,Quantity Card_Name 0 ...,117.57,85.81,53,0,176
11,Mardu Midrange,Quantity Card_Name 0...,420.91,429.45,50,4,162
8,Jund Midrange,Quantity Card_Name 0...,467.72,498.38,48,7,144
9,Kami War,Quantity Card_Name 0 ...,358.16,678.11,42,9,140
2,5 Color Midrange,Quantity Card_Name 0 ...,505.99,759.49,42,6,130
7,Selesnya Poison,Quantity Card_Name 0 ...,214.62,240.2,46,0,122
0,Grixis Midrange,Quantity Card_Name 0 ...,384.53,419.47,43,3,120


Finally, we output the data to a csv for visualization in Tableau.

In [22]:
decks[['Deck_Name', 'Paper_Price', 'MTGO_Price_Tix', 'MTGArena_Price']].to_csv("Format Price Data/Standard_Metagame_Deck_Prices.csv",index=False) #no need to include decklists

The final Data Viz can be found at https://public.tableau.com/app/profile/jack.garn/viz/MTGStandardDeckPrices/Dashboard1?publish=yes